In [98]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import Imputer
%matplotlib inline

In [99]:
data = pd.read_csv('../Data/Amazon.csv')
# data = data[0:1000]
data.shape
# data.head(5)

(455000, 13)

In [100]:
# define y
y = data.iloc[:, 12].values
y.shape

(455000,)

In [101]:
#bag of words on text // Replace with hashingvectorizer
# vectorize Bag of Words from review text; as sparse matrix
from sklearn.feature_extraction.text import HashingVectorizer
hv = HashingVectorizer(n_features=2 ** 17, non_negative=True)
XText = hv.transform(data.Text)
XText.shape

(455000, 131072)

In [102]:
#bag of words on summary
# creating newSummary with "no summary given" where summary does not exist.
# data['SummaryFill'] = np.where(pd.isnull(data['Summary']) == True, 'no summary given', data['Summary'])

data['Summary'].fillna('null', inplace=True)
XSummary = hv.transform(data.Summary)
XSummary.shape

(455000, 131072)

In [140]:
# Ngrams
from sklearn.feature_extraction.text import TfidfVectorizer
# nVec = TfidfVectorizer(ngram_range=(3,3))
nVec = TfidfVectorizer(analyzer='word', ngram_range=(2, 3), min_df=1)
Xngram = nVec.fit_transform(data.Text)


In [141]:
type(Xngram)

scipy.sparse.csr.csr_matrix

In [142]:
# features from Amazon.csv to add to feature set
# XScore = data.iloc[:, 7].values.reshape(data.shape[0], 1)

# data['reviewLen'] = data['Text'].str.len()
# data['summaryLen'] = data['Summary'].str.len()
# data['usernameWords'] = data ['ProfileName'].str.split().str.len()
# data['newLines'] = data['Text'].str.split('\n').str.len()

# data['hasSC'] = data['Text'].str.count(';')
# data['hasEX'] = data['Text'].str.count('!')
# data['hasQ'] = data['Text'].str.count('\?')
# data['punctCount'] = data['Text'].str.count('[.,!;:()/\?-]')
# data['punctToWords'] = data['punctCount'] / data['reviewLen']
# data['avWordLength'] = data['Text'].str.len() // (data['Text'].str.count(' ') + 1)
# data['sumLen'] = data['Summary'].str.count('\S')
# data['avSumWordLen'] = data['Summary'].str.len() // (data['Summary'].str.count(' ') + 1)
# data['sumHasEX'] = data['Summary'].str.count('!')
# data['sumHasQ'] = data['Summary'].str.count('\?')

# XreviewLen = data.iloc[:, 13].values.reshape(data.shape[0], 1)
# XsummaryLen = data.iloc[:, 14].values.reshape(data.shape[0], 1)
# XusernameWords = data.iloc[:, 15].values.reshape(data.shape[0], 1)
# XexclamationNumber = data.iloc[:, 16].values.reshape(data.shape[0], 1)
# XnewLines = data.iloc[:, 17].values.reshape(data.shape[0], 1)
# XhasSC = data.iloc[:, 15].values.reshape(data.shape[0], 1)
# XhasEX = data.iloc[:, 16].values.reshape(data.shape[0], 1)
# XhasQ = data.iloc[:, 17].values.reshape(data.shape[0], 1)
# XpunctCount = data.iloc[:, 18].values.reshape(data.shape[0], 1)
# XpunctToWords = data.iloc[:, 19].values.reshape(data.shape[0], 1)
# XavWordLength = data.iloc[:, 20].values.reshape(data.shape[0], 1)
# XsumLen = data.iloc[:, 21].values.reshape(data.shape[0], 1)
# XavSumWordLen = data.iloc[:, 22].values.reshape(data.shape[0], 1)
# XsumHasEX = data.iloc[:, 23].values.reshape(data.shape[0], 1)
# XsumHasQ = data.iloc[:, 24].values.reshape(data.shape[0], 1)

# Xtoadd = np.concatenate((XScore,XreviewLen,XsummaryLen,XusernameWords,XnewLines,XhasSC,XhasEX,XhasQ,XpunctCount,XpunctToWords,XsumLen,XavSumWordLen,XsumHasEX,XsumHasQ), axis=1)


In [143]:
# vectorize Bag of Words from Summary text; as sparse matrix
from sklearn.feature_extraction.text import HashingVectorizer
hv = HashingVectorizer(n_features=2 ** 17, non_negative=True)
XhashText = hv.transform(data.Text)
XhashSummary = hv.transform(data.Summary)

In [144]:
# convert to CSR
from scipy.sparse import csr_matrix, hstack
XtoaddSparse = csr_matrix(Xtoadd)
# Xfinal = hstack([ Xtoadd, Xngram, XtoaddSparse,XhashText,XhashSummary,XText,XSummary])
Xfinal = hstack([ Xngram, XtoaddSparse,XhashText,XhashSummary,XText,XSummary])
X = csr_matrix(Xfinal)
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
X = imp.fit_transform(X)

In [145]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.3, random_state=1)

In [146]:
# report on training and test sets
def print_results():
    print('Error rate on training set: ')
    print((y_train != y_pred).sum() / X_train.shape[0])
    print('Accuracy rate on training set: ')
    print(1 - (y_train != y_pred).sum() / X_train.shape[0])
    print('True positive rate on training set:')
    print(((y_train==True) & (y_pred==True)).sum() / y_train.sum())
    print('**************')
    print('Error rate on test set: ')
    print((y_test != y_pred_test).sum() / X_test.shape[0])
    print('Accuracy rate on test set: ')
    print(1 - (y_test != y_pred_test).sum() / X_test.shape[0])
    print('True positive rate on test set')
    print(((y_test==True) & (y_pred_test==True)).sum() / y_test.sum())
    print('True negative rate on test set')
    print(((y_test==False) & (y_pred_test==False)).sum() / (y_test.shape[0] - y_test.sum()))

In [147]:
# feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler(with_mean=False)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [148]:
# Perceptron
from sklearn import linear_model
clf = linear_model.SGDClassifier(loss='squared_hinge')
clf.fit(X_train_std, y_train)
y_pred = clf.fit(X_train_std, y_train).predict(X_train_std)
y_pred_test = clf.predict(X_test_std)
print_results()

Error rate on training set: 
0.000835164835165
Accuracy rate on training set: 
0.999164835165
True positive rate on training set:
0.993858707322
**************
Error rate on test set: 
0.134490842491
Accuracy rate on test set: 
0.865509157509
True positive rate on test set
0.446834170854
True negative rate on test set
0.898427499012


In [149]:
# from sklearn.svm import SVC
# svm = SVC(kernel='linear', C=1.0, random_state=0)
# svm.fit(X_train, y_train)
# y_pred = svm.predict(X_train)
# y_pred_test = svm.predict(X_test)
# print_results()